In [85]:
#importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [6]:
df=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\train_Df64byy.csv')

In [7]:
df1=df.copy()

In [4]:
df.isnull().sum()

ID                             0
City_Code                      0
Region_Code                    0
Accomodation_Type              0
Reco_Insurance_Type            0
Upper_Age                      0
Lower_Age                      0
Is_Spouse                      0
Health Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
Reco_Policy_Cat                0
Reco_Policy_Premium            0
Response                       0
dtype: int64

In [5]:
df.T.apply(lambda x: x.nunique(), axis=1)

ID                         50882
City_Code                     36
Region_Code                 5316
Accomodation_Type              2
Reco_Insurance_Type            2
Upper_Age                     58
Lower_Age                     60
Is_Spouse                      2
Health Indicator               9
Holding_Policy_Duration       15
Holding_Policy_Type            4
Reco_Policy_Cat               22
Reco_Policy_Premium         6977
Response                       2
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
ID                         50882 non-null int64
City_Code                  50882 non-null object
Region_Code                50882 non-null int64
Accomodation_Type          50882 non-null object
Reco_Insurance_Type        50882 non-null object
Upper_Age                  50882 non-null int64
Lower_Age                  50882 non-null int64
Is_Spouse                  50882 non-null object
Health Indicator           39191 non-null object
Holding_Policy_Duration    30631 non-null object
Holding_Policy_Type        30631 non-null float64
Reco_Policy_Cat            50882 non-null int64
Reco_Policy_Premium        50882 non-null float64
Response                   50882 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 5.4+ MB


In [8]:
df['Holding_Policy_Duration']=df['Holding_Policy_Duration'].replace('14+',15)

In [9]:
df=df.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category'})

In [10]:
#No missing values in num variables
num_vars=df.columns[df.dtypes!='category']
cat_vars=df.columns[df.dtypes=='category']

# MISSING VALUE ANALYSIS & IMPUTATION

In [11]:
df['Health Indicator']=df['Health Indicator'].cat.codes
df['Health Indicator']=df['Health Indicator'].replace(-1, np.nan)
df['Accomodation_Type']=df['Accomodation_Type'].cat.codes
df['Reco_Insurance_Type']=df['Reco_Insurance_Type'].cat.codes
df['Is_Spouse']=df['Is_Spouse'].cat.codes
df['City_Code']=df['City_Code'].cat.codes

# df1['Health Indicator'].isnull().sum()

In [12]:
df2=df.copy()

In [13]:
my_imputer = SimpleImputer()
data_with_imputed_values = pd.DataFrame(my_imputer.fit_transform(df2))

In [14]:
data_with_imputed_values.columns=df2.columns
data_with_imputed_values.index=df2.index

# OUTLIER ANALYSIS

In [15]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [16]:
df2.shape

(50882, 14)

In [17]:
data_with_imputed_values=remove_outlier(data_with_imputed_values,'Reco_Policy_Premium')

In [18]:
data_with_imputed_values.shape

(50061, 14)

# standardization

In [19]:
data_with_imputed_values[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(data_with_imputed_values[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

# Splitting of Dataset

In [20]:
d1=data_with_imputed_values[:500]
data_with_imputed_values=data_with_imputed_values[500:]

In [21]:
X=data_with_imputed_values.loc[:, data_with_imputed_values.columns != 'ID']
y=data_with_imputed_values['Response']
X.drop(columns=['Response'],inplace=True)

C:\Python\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
data_with_imputed_values.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Response'],
      dtype='object')

In [23]:
#Best set of columns
X=X[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat','Reco_Policy_Premium','Reco_Insurance_Type']]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.001, random_state=100)

In [26]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(y_train)
test_y = encoder.fit_transform(y_test)

In [27]:
smote = SMOTE(k_neighbors=5, n_jobs=-1,random_state=777)
smote_enn = make_pipeline(SimpleImputer(), SMOTEENN(smote=smote))
sm_xtrain, sm_ytrain = smote_enn.fit_resample(X, y)

# RANDOM FOREST

In [176]:
from sklearn.ensemble import RandomForestClassifier
rndmf = SelectFromModel(RandomForestClassifier(n_estimators=500,class_weight='balanced'))
rndmf.fit(sm_xtrain, sm_ytrain)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight='balanced',
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=500, n_jobs=None,
                                                 oob_score=

In [177]:
selected_feat= X_train.columns[(rndmf.get_support())]
print(selected_feat)

Index(['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat',
       'Reco_Policy_Premium'],
      dtype='object')


In [178]:
# Results from Grid Search 

#  The best estimator across ALL searched params:
#  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
#                        criterion='gini', max_depth=30, max_features='auto',
#                        max_leaf_nodes=None, max_samples=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=500,
#                        n_jobs=None, oob_score=False, random_state=None,
#                        verbose=0, warm_start=False)

#  The best score across ALL searched params:
#  0.837041945489787

#  The best parameters across ALL searched params:
#  {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}

In [28]:
RF_model= RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
RF_model=RF_model.fit(sm_xtrain, sm_ytrain)
y_pred1=RF_model.predict(X_test)

In [25]:
y_pred1_proba=RF_model.predict_proba(X_test)

In [30]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred1)

0.787941787941788

In [31]:
report = classification_report(test_y,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.73      0.82        37
           1       0.52      0.85      0.65        13

    accuracy                           0.76        50
   macro avg       0.73      0.79      0.73        50
weighted avg       0.83      0.76      0.77        50



In [32]:
Conf_metrics_sm_rndm = metrics.confusion_matrix(test_y, y_pred1,labels=[0,1])
print(Conf_metrics_sm_rndm)

[[27 10]
 [ 2 11]]


# LIGHT GBM

In [86]:
from lightgbm import LGBMClassifier
LightGBM=LGBMClassifier(random_state=2)
LightGBM.fit(sm_xtrain,sm_ytrain)
y_pred5=LightGBM.predict(X_test)

In [87]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred5)

0.632016632016632

In [32]:
# fit_params={"early_stopping_rounds":30, 
#             "eval_metric" : 'auc', 
#             "eval_set" : [(X_test,y_test)],
#             'eval_names': ['valid'],
#             #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
#             'verbose': 100,
#             'categorical_feature': 'auto'}

In [29]:
# from scipy.stats import randint as sp_randint
# from scipy.stats import uniform as sp_uniform
# param_test ={'num_leaves': sp_randint(6, 50), 
#              'min_child_samples': sp_randint(100, 500), 
#              'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#              'subsample': sp_uniform(loc=0.2, scale=0.8), 
#              'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
#              'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
#              'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [30]:
# #This parameter defines the number of HP points to be tested
# n_HP_points_to_test = 100

# import lightgbm as lgb
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# #n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
# clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
# gs = RandomizedSearchCV(
#     estimator=clf, param_distributions=param_test, 
#     n_iter=n_HP_points_to_test,
#     scoring='roc_auc',
#     cv=3,
#     refit=True,
#     random_state=314,
#     verbose=True)


In [35]:
opt_parameters= {'colsample_bytree': 0.6847074798584072, 'min_child_samples': 125, 'min_child_weight': 10.0, 'num_leaves': 49, 'reg_alpha': 5, 'reg_lambda': 50, 'subsample': 0.20892704149103666}

In [38]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
clf_sw = lgb.LGBMClassifier(**clf.get_params())
#set optimal parameters
clf_sw.set_params(**opt_parameters)

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.6847074798584072, importance_type='split',
               learning_rate=0.1, max_depth=-1, metric='None',
               min_child_samples=125, min_child_weight=10.0, min_split_gain=0.0,
               n_estimators=5000, n_jobs=4, num_leaves=49, objective=None,
               random_state=314, reg_alpha=5, reg_lambda=50, silent=True,
               subsample=0.20892704149103666, subsample_for_bin=200000,
               subsample_freq=0)

In [39]:
gs=clf_sw.fit(sm_xtrain, sm_ytrain)

In [40]:
y_pred6 = gs.predict(X_test)

In [33]:
# gs.fit(sm_xtrain, sm_ytrain, **fit_params)
# print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2]	valid's auc: 0.637214
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[9]	valid's auc: 0.674636


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.682952
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2]	valid's auc: 0.672557
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.7079
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid's auc: 0.691268
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	valid's auc: 0.694387
Training until validation scores don't i

Early stopping, best iteration is:
[31]	valid's auc: 0.619543
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[15]	valid's auc: 0.639293
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid's auc: 0.636175
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.683992
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[31]	valid's auc: 0.68815
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid's auc: 0.5
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2

Early stopping, best iteration is:
[2]	valid's auc: 0.675676
Training until validation scores don't improve for 30 rounds
[100]	valid's auc: 0.675676
Early stopping, best iteration is:
[71]	valid's auc: 0.701663
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[6]	valid's auc: 0.611227
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[55]	valid's auc: 0.659044
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[40]	valid's auc: 0.659044
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[28]	valid's auc: 0.60499
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[51]	valid's auc: 0.719335
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[43]	valid's auc: 0.681913
Training until validation scores don't improve for 3

Training until validation scores don't improve for 30 rounds
[100]	valid's auc: 0.648649
Early stopping, best iteration is:
[74]	valid's auc: 0.671518
Training until validation scores don't improve for 30 rounds
[100]	valid's auc: 0.623701
Early stopping, best iteration is:
[76]	valid's auc: 0.656965
Training until validation scores don't improve for 30 rounds
[100]	valid's auc: 0.640333
Early stopping, best iteration is:
[75]	valid's auc: 0.663202
Training until validation scores don't improve for 30 rounds
[100]	valid's auc: 0.659044
Early stopping, best iteration is:
[104]	valid's auc: 0.665281
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[66]	valid's auc: 0.627859
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[64]	valid's auc: 0.634096
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[53]	valid's auc: 0.642412
Training until validation 

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[4]	valid's auc: 0.686071
Training until validation scores don't improve for 30 rounds
[100]	valid's auc: 0.64657
Early stopping, best iteration is:
[73]	valid's auc: 0.669439
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2]	valid's auc: 0.650728
Training until validation scores don't improve for 30 rounds
[100]	valid's auc: 0.669439
Early stopping, best iteration is:
[77]	valid's auc: 0.673597
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[16]	valid's auc: 0.663202
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2]	valid's auc: 0.634096
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[12]	valid's auc: 0.634096
Training until validation scores don't improve for 30 rounds
Early stopping, best itera

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   37.9s finished


[100]	valid's auc: 0.696466
[200]	valid's auc: 0.756757
Early stopping, best iteration is:
[188]	valid's auc: 0.760915
Best score reached: 0.8930842127162499 with params: {'colsample_bytree': 0.6847074798584072, 'min_child_samples': 125, 'min_child_weight': 10.0, 'num_leaves': 49, 'reg_alpha': 5, 'reg_lambda': 50, 'subsample': 0.20892704149103666} 


In [41]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred6)

0.7785862785862786

In [42]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred5)

0.632016632016632

# Decison tree

In [43]:
from sklearn.tree import DecisionTreeClassifier
dt_cls = DecisionTreeClassifier(random_state=101)
dt_cls.fit(sm_xtrain,sm_ytrain)
y_pred2=dt_cls.predict(X_test)                                           

In [44]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred2)

0.9074844074844075

In [45]:
report = classification_report(test_y,y_pred2)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.89      0.93        37
           1       0.75      0.92      0.83        13

    accuracy                           0.90        50
   macro avg       0.86      0.91      0.88        50
weighted avg       0.91      0.90      0.90        50



In [35]:
Conf_metrics_sm_rndm = metrics.confusion_matrix(test_y, y_pred2,labels=[0,1])
print(Conf_metrics_sm_rndm)

[[30  7]
 [ 2 11]]


# CAT BOOST

In [46]:
CatBoost=CatBoostClassifier(random_state=101)                                                                                                   
CatBoost.fit(sm_xtrain,sm_ytrain)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 66.4ms	remaining: 1m 6s
1:	learn: 0.6679542	total: 76.5ms	remaining: 38.2s
2:	learn: 0.6586634	total: 85.4ms	remaining: 28.4s
3:	learn: 0.6506448	total: 95.1ms	remaining: 23.7s
4:	learn: 0.6408058	total: 106ms	remaining: 21.2s
5:	learn: 0.6340672	total: 118ms	remaining: 19.5s
6:	learn: 0.6286264	total: 131ms	remaining: 18.5s
7:	learn: 0.6226329	total: 143ms	remaining: 17.7s
8:	learn: 0.6184221	total: 155ms	remaining: 17s
9:	learn: 0.6140516	total: 166ms	remaining: 16.4s
10:	learn: 0.6095748	total: 178ms	remaining: 16s
11:	learn: 0.6056595	total: 197ms	remaining: 16.2s
12:	learn: 0.6009331	total: 212ms	remaining: 16.1s
13:	learn: 0.5953545	total: 224ms	remaining: 15.8s
14:	learn: 0.5916079	total: 238ms	remaining: 15.6s
15:	learn: 0.5871947	total: 251ms	remaining: 15.4s
16:	learn: 0.5799814	total: 264ms	remaining: 15.3s
17:	learn: 0.5771092	total: 277ms	remaining: 15.1s
18:	learn: 0.5719313	total: 291ms	remaining: 15s
19:	learn: 0.5

166:	learn: 0.4203855	total: 2.73s	remaining: 13.6s
167:	learn: 0.4200934	total: 2.74s	remaining: 13.6s
168:	learn: 0.4197580	total: 2.75s	remaining: 13.6s
169:	learn: 0.4189651	total: 2.77s	remaining: 13.5s
170:	learn: 0.4186179	total: 2.79s	remaining: 13.5s
171:	learn: 0.4181877	total: 2.8s	remaining: 13.5s
172:	learn: 0.4178532	total: 2.82s	remaining: 13.5s
173:	learn: 0.4176437	total: 2.83s	remaining: 13.4s
174:	learn: 0.4173844	total: 2.85s	remaining: 13.4s
175:	learn: 0.4170745	total: 2.86s	remaining: 13.4s
176:	learn: 0.4167306	total: 2.88s	remaining: 13.4s
177:	learn: 0.4163079	total: 2.89s	remaining: 13.3s
178:	learn: 0.4158617	total: 2.9s	remaining: 13.3s
179:	learn: 0.4154727	total: 2.92s	remaining: 13.3s
180:	learn: 0.4152674	total: 2.94s	remaining: 13.3s
181:	learn: 0.4148733	total: 2.96s	remaining: 13.3s
182:	learn: 0.4143350	total: 2.99s	remaining: 13.4s
183:	learn: 0.4140410	total: 3.02s	remaining: 13.4s
184:	learn: 0.4135355	total: 3.04s	remaining: 13.4s
185:	learn: 0.

332:	learn: 0.3572630	total: 5.79s	remaining: 11.6s
333:	learn: 0.3571139	total: 5.82s	remaining: 11.6s
334:	learn: 0.3569167	total: 5.84s	remaining: 11.6s
335:	learn: 0.3565053	total: 5.86s	remaining: 11.6s
336:	learn: 0.3563696	total: 5.88s	remaining: 11.6s
337:	learn: 0.3560269	total: 5.9s	remaining: 11.6s
338:	learn: 0.3556117	total: 5.92s	remaining: 11.5s
339:	learn: 0.3553019	total: 5.94s	remaining: 11.5s
340:	learn: 0.3551200	total: 5.96s	remaining: 11.5s
341:	learn: 0.3549500	total: 5.97s	remaining: 11.5s
342:	learn: 0.3545114	total: 5.99s	remaining: 11.5s
343:	learn: 0.3542186	total: 6s	remaining: 11.5s
344:	learn: 0.3539000	total: 6.02s	remaining: 11.4s
345:	learn: 0.3535411	total: 6.05s	remaining: 11.4s
346:	learn: 0.3534507	total: 6.06s	remaining: 11.4s
347:	learn: 0.3530517	total: 6.08s	remaining: 11.4s
348:	learn: 0.3528620	total: 6.09s	remaining: 11.4s
349:	learn: 0.3525342	total: 6.11s	remaining: 11.4s
350:	learn: 0.3523856	total: 6.13s	remaining: 11.3s
351:	learn: 0.35

492:	learn: 0.3252503	total: 8.5s	remaining: 8.74s
493:	learn: 0.3251172	total: 8.51s	remaining: 8.72s
494:	learn: 0.3250046	total: 8.53s	remaining: 8.7s
495:	learn: 0.3249080	total: 8.54s	remaining: 8.68s
496:	learn: 0.3247105	total: 8.56s	remaining: 8.66s
497:	learn: 0.3245770	total: 8.58s	remaining: 8.65s
498:	learn: 0.3244784	total: 8.59s	remaining: 8.63s
499:	learn: 0.3243497	total: 8.61s	remaining: 8.61s
500:	learn: 0.3242095	total: 8.63s	remaining: 8.6s
501:	learn: 0.3240897	total: 8.65s	remaining: 8.58s
502:	learn: 0.3239015	total: 8.66s	remaining: 8.56s
503:	learn: 0.3237987	total: 8.68s	remaining: 8.54s
504:	learn: 0.3236880	total: 8.69s	remaining: 8.52s
505:	learn: 0.3235400	total: 8.71s	remaining: 8.51s
506:	learn: 0.3234257	total: 8.73s	remaining: 8.49s
507:	learn: 0.3233202	total: 8.75s	remaining: 8.47s
508:	learn: 0.3231908	total: 8.76s	remaining: 8.45s
509:	learn: 0.3230773	total: 8.78s	remaining: 8.43s
510:	learn: 0.3229635	total: 8.79s	remaining: 8.42s
511:	learn: 0.3

652:	learn: 0.3057909	total: 11s	remaining: 5.85s
653:	learn: 0.3056714	total: 11s	remaining: 5.84s
654:	learn: 0.3055410	total: 11.1s	remaining: 5.82s
655:	learn: 0.3054457	total: 11.1s	remaining: 5.8s
656:	learn: 0.3053621	total: 11.1s	remaining: 5.79s
657:	learn: 0.3052218	total: 11.1s	remaining: 5.77s
658:	learn: 0.3051549	total: 11.1s	remaining: 5.76s
659:	learn: 0.3050008	total: 11.1s	remaining: 5.74s
660:	learn: 0.3048823	total: 11.2s	remaining: 5.72s
661:	learn: 0.3048124	total: 11.2s	remaining: 5.71s
662:	learn: 0.3046744	total: 11.2s	remaining: 5.69s
663:	learn: 0.3046230	total: 11.2s	remaining: 5.67s
664:	learn: 0.3045188	total: 11.2s	remaining: 5.66s
665:	learn: 0.3044218	total: 11.2s	remaining: 5.64s
666:	learn: 0.3043253	total: 11.3s	remaining: 5.62s
667:	learn: 0.3042170	total: 11.3s	remaining: 5.61s
668:	learn: 0.3041603	total: 11.3s	remaining: 5.59s
669:	learn: 0.3041097	total: 11.3s	remaining: 5.57s
670:	learn: 0.3040226	total: 11.3s	remaining: 5.55s
671:	learn: 0.303

815:	learn: 0.2907099	total: 13.7s	remaining: 3.09s
816:	learn: 0.2905168	total: 13.7s	remaining: 3.08s
817:	learn: 0.2903982	total: 13.8s	remaining: 3.06s
818:	learn: 0.2902975	total: 13.8s	remaining: 3.04s
819:	learn: 0.2902293	total: 13.8s	remaining: 3.03s
820:	learn: 0.2901360	total: 13.8s	remaining: 3.01s
821:	learn: 0.2900767	total: 13.8s	remaining: 2.99s
822:	learn: 0.2899938	total: 13.8s	remaining: 2.98s
823:	learn: 0.2899065	total: 13.9s	remaining: 2.96s
824:	learn: 0.2898437	total: 13.9s	remaining: 2.94s
825:	learn: 0.2897834	total: 13.9s	remaining: 2.92s
826:	learn: 0.2897266	total: 13.9s	remaining: 2.91s
827:	learn: 0.2895873	total: 13.9s	remaining: 2.89s
828:	learn: 0.2895197	total: 13.9s	remaining: 2.87s
829:	learn: 0.2893973	total: 13.9s	remaining: 2.86s
830:	learn: 0.2891751	total: 14s	remaining: 2.84s
831:	learn: 0.2891185	total: 14s	remaining: 2.82s
832:	learn: 0.2890525	total: 14s	remaining: 2.8s
833:	learn: 0.2889932	total: 14s	remaining: 2.79s
834:	learn: 0.2889123

975:	learn: 0.2778203	total: 16.5s	remaining: 405ms
976:	learn: 0.2777686	total: 16.5s	remaining: 388ms
977:	learn: 0.2776804	total: 16.5s	remaining: 371ms
978:	learn: 0.2775982	total: 16.5s	remaining: 355ms
979:	learn: 0.2774787	total: 16.6s	remaining: 338ms
980:	learn: 0.2774271	total: 16.6s	remaining: 321ms
981:	learn: 0.2773683	total: 16.6s	remaining: 304ms
982:	learn: 0.2772605	total: 16.6s	remaining: 287ms
983:	learn: 0.2771929	total: 16.6s	remaining: 271ms
984:	learn: 0.2771370	total: 16.7s	remaining: 254ms
985:	learn: 0.2770458	total: 16.7s	remaining: 237ms
986:	learn: 0.2769699	total: 16.7s	remaining: 220ms
987:	learn: 0.2769228	total: 16.7s	remaining: 203ms
988:	learn: 0.2768567	total: 16.8s	remaining: 186ms
989:	learn: 0.2768206	total: 16.8s	remaining: 169ms
990:	learn: 0.2767359	total: 16.8s	remaining: 152ms
991:	learn: 0.2766794	total: 16.8s	remaining: 136ms
992:	learn: 0.2766293	total: 16.8s	remaining: 119ms
993:	learn: 0.2765418	total: 16.8s	remaining: 102ms
994:	learn: 

In [47]:
y_pred3=CatBoost.predict(np.array(X_test))

In [48]:
#reco insurance type
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y,  y_pred3)

0.6746361746361745

In [273]:
Conf_metrics_sm_cat = metrics.confusion_matrix(test_y, y_pred3,labels=[0,1])
print(Conf_metrics_sm_cat)

[[30  7]
 [ 7  6]]


# ENSEMBLE

In [51]:
from sklearn.ensemble import VotingClassifier
model6 = VotingClassifier(estimators=[('Cb', CatBoost),('dt', dt_cls),('rf', RF_model)], voting='soft')
model6.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_6=model6.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 28ms	remaining: 28s
1:	learn: 0.6679542	total: 49.7ms	remaining: 24.8s
2:	learn: 0.6586634	total: 65.8ms	remaining: 21.9s
3:	learn: 0.6506448	total: 82.9ms	remaining: 20.6s
4:	learn: 0.6408058	total: 96.4ms	remaining: 19.2s
5:	learn: 0.6340672	total: 111ms	remaining: 18.3s
6:	learn: 0.6286264	total: 125ms	remaining: 17.8s
7:	learn: 0.6226329	total: 140ms	remaining: 17.4s
8:	learn: 0.6184221	total: 153ms	remaining: 16.9s
9:	learn: 0.6140516	total: 169ms	remaining: 16.7s
10:	learn: 0.6095748	total: 182ms	remaining: 16.4s
11:	learn: 0.6056595	total: 196ms	remaining: 16.2s
12:	learn: 0.6009331	total: 210ms	remaining: 15.9s
13:	learn: 0.5953545	total: 223ms	remaining: 15.7s
14:	learn: 0.5916079	total: 236ms	remaining: 15.5s
15:	learn: 0.5871947	total: 250ms	remaining: 15.4s
16:	learn: 0.5799814	total: 265ms	remaining: 15.3s
17:	learn: 0.5771092	total: 280ms	remaining: 15.3s
18:	learn: 0.5719313	total: 297ms	remaining: 15.3s
19:	learn: 

160:	learn: 0.4224735	total: 2.79s	remaining: 14.5s
161:	learn: 0.4218811	total: 2.8s	remaining: 14.5s
162:	learn: 0.4215547	total: 2.84s	remaining: 14.6s
163:	learn: 0.4211132	total: 2.87s	remaining: 14.6s
164:	learn: 0.4209344	total: 2.89s	remaining: 14.6s
165:	learn: 0.4207196	total: 2.92s	remaining: 14.7s
166:	learn: 0.4203855	total: 2.93s	remaining: 14.6s
167:	learn: 0.4200934	total: 2.96s	remaining: 14.7s
168:	learn: 0.4197580	total: 2.99s	remaining: 14.7s
169:	learn: 0.4189651	total: 3.02s	remaining: 14.7s
170:	learn: 0.4186179	total: 3.04s	remaining: 14.7s
171:	learn: 0.4181877	total: 3.07s	remaining: 14.8s
172:	learn: 0.4178532	total: 3.08s	remaining: 14.7s
173:	learn: 0.4176437	total: 3.1s	remaining: 14.7s
174:	learn: 0.4173844	total: 3.12s	remaining: 14.7s
175:	learn: 0.4170745	total: 3.15s	remaining: 14.8s
176:	learn: 0.4167306	total: 3.17s	remaining: 14.7s
177:	learn: 0.4163079	total: 3.2s	remaining: 14.8s
178:	learn: 0.4158617	total: 3.22s	remaining: 14.8s
179:	learn: 0.4

319:	learn: 0.3613531	total: 5.92s	remaining: 12.6s
320:	learn: 0.3611573	total: 5.94s	remaining: 12.6s
321:	learn: 0.3608421	total: 5.96s	remaining: 12.5s
322:	learn: 0.3602778	total: 5.98s	remaining: 12.5s
323:	learn: 0.3600014	total: 6s	remaining: 12.5s
324:	learn: 0.3598087	total: 6.03s	remaining: 12.5s
325:	learn: 0.3596011	total: 6.05s	remaining: 12.5s
326:	learn: 0.3591945	total: 6.06s	remaining: 12.5s
327:	learn: 0.3589449	total: 6.08s	remaining: 12.5s
328:	learn: 0.3587682	total: 6.1s	remaining: 12.4s
329:	learn: 0.3582579	total: 6.12s	remaining: 12.4s
330:	learn: 0.3578227	total: 6.14s	remaining: 12.4s
331:	learn: 0.3574625	total: 6.16s	remaining: 12.4s
332:	learn: 0.3572630	total: 6.18s	remaining: 12.4s
333:	learn: 0.3571139	total: 6.19s	remaining: 12.4s
334:	learn: 0.3569167	total: 6.21s	remaining: 12.3s
335:	learn: 0.3565053	total: 6.23s	remaining: 12.3s
336:	learn: 0.3563696	total: 6.25s	remaining: 12.3s
337:	learn: 0.3560269	total: 6.26s	remaining: 12.3s
338:	learn: 0.35

481:	learn: 0.3268897	total: 8.82s	remaining: 9.48s
482:	learn: 0.3267729	total: 8.84s	remaining: 9.46s
483:	learn: 0.3266848	total: 8.86s	remaining: 9.45s
484:	learn: 0.3265671	total: 8.89s	remaining: 9.44s
485:	learn: 0.3264263	total: 8.92s	remaining: 9.43s
486:	learn: 0.3261930	total: 8.94s	remaining: 9.42s
487:	learn: 0.3260570	total: 8.98s	remaining: 9.42s
488:	learn: 0.3257571	total: 9s	remaining: 9.4s
489:	learn: 0.3256668	total: 9.02s	remaining: 9.39s
490:	learn: 0.3255310	total: 9.05s	remaining: 9.39s
491:	learn: 0.3254191	total: 9.08s	remaining: 9.37s
492:	learn: 0.3252503	total: 9.1s	remaining: 9.36s
493:	learn: 0.3251172	total: 9.13s	remaining: 9.35s
494:	learn: 0.3250046	total: 9.15s	remaining: 9.33s
495:	learn: 0.3249080	total: 9.18s	remaining: 9.33s
496:	learn: 0.3247105	total: 9.2s	remaining: 9.31s
497:	learn: 0.3245770	total: 9.22s	remaining: 9.29s
498:	learn: 0.3244784	total: 9.24s	remaining: 9.28s
499:	learn: 0.3243497	total: 9.26s	remaining: 9.26s
500:	learn: 0.3242

646:	learn: 0.3064081	total: 12s	remaining: 6.55s
647:	learn: 0.3062515	total: 12s	remaining: 6.53s
648:	learn: 0.3061605	total: 12s	remaining: 6.51s
649:	learn: 0.3060862	total: 12.1s	remaining: 6.5s
650:	learn: 0.3059759	total: 12.1s	remaining: 6.48s
651:	learn: 0.3058707	total: 12.1s	remaining: 6.46s
652:	learn: 0.3057909	total: 12.1s	remaining: 6.45s
653:	learn: 0.3056714	total: 12.2s	remaining: 6.43s
654:	learn: 0.3055410	total: 12.2s	remaining: 6.42s
655:	learn: 0.3054457	total: 12.2s	remaining: 6.39s
656:	learn: 0.3053621	total: 12.2s	remaining: 6.38s
657:	learn: 0.3052218	total: 12.2s	remaining: 6.36s
658:	learn: 0.3051549	total: 12.2s	remaining: 6.34s
659:	learn: 0.3050008	total: 12.3s	remaining: 6.32s
660:	learn: 0.3048823	total: 12.3s	remaining: 6.3s
661:	learn: 0.3048124	total: 12.3s	remaining: 6.28s
662:	learn: 0.3046744	total: 12.3s	remaining: 6.26s
663:	learn: 0.3046230	total: 12.3s	remaining: 6.24s
664:	learn: 0.3045188	total: 12.4s	remaining: 6.23s
665:	learn: 0.304421

811:	learn: 0.2910277	total: 15.1s	remaining: 3.49s
812:	learn: 0.2909515	total: 15.1s	remaining: 3.48s
813:	learn: 0.2908974	total: 15.1s	remaining: 3.46s
814:	learn: 0.2907856	total: 15.1s	remaining: 3.44s
815:	learn: 0.2907099	total: 15.2s	remaining: 3.42s
816:	learn: 0.2905168	total: 15.2s	remaining: 3.4s
817:	learn: 0.2903982	total: 15.2s	remaining: 3.38s
818:	learn: 0.2902975	total: 15.2s	remaining: 3.36s
819:	learn: 0.2902293	total: 15.2s	remaining: 3.34s
820:	learn: 0.2901360	total: 15.2s	remaining: 3.32s
821:	learn: 0.2900767	total: 15.3s	remaining: 3.3s
822:	learn: 0.2899938	total: 15.3s	remaining: 3.29s
823:	learn: 0.2899065	total: 15.3s	remaining: 3.27s
824:	learn: 0.2898437	total: 15.3s	remaining: 3.25s
825:	learn: 0.2897834	total: 15.3s	remaining: 3.23s
826:	learn: 0.2897266	total: 15.3s	remaining: 3.21s
827:	learn: 0.2895873	total: 15.4s	remaining: 3.19s
828:	learn: 0.2895197	total: 15.4s	remaining: 3.17s
829:	learn: 0.2893973	total: 15.4s	remaining: 3.15s
830:	learn: 0.

970:	learn: 0.2782399	total: 18.2s	remaining: 544ms
971:	learn: 0.2781542	total: 18.2s	remaining: 525ms
972:	learn: 0.2780641	total: 18.3s	remaining: 507ms
973:	learn: 0.2779606	total: 18.3s	remaining: 488ms
974:	learn: 0.2778747	total: 18.3s	remaining: 469ms
975:	learn: 0.2778203	total: 18.3s	remaining: 450ms
976:	learn: 0.2777686	total: 18.3s	remaining: 432ms
977:	learn: 0.2776804	total: 18.4s	remaining: 413ms
978:	learn: 0.2775982	total: 18.4s	remaining: 394ms
979:	learn: 0.2774787	total: 18.4s	remaining: 375ms
980:	learn: 0.2774271	total: 18.4s	remaining: 357ms
981:	learn: 0.2773683	total: 18.4s	remaining: 338ms
982:	learn: 0.2772605	total: 18.4s	remaining: 319ms
983:	learn: 0.2771929	total: 18.5s	remaining: 300ms
984:	learn: 0.2771370	total: 18.5s	remaining: 281ms
985:	learn: 0.2770458	total: 18.5s	remaining: 263ms
986:	learn: 0.2769699	total: 18.5s	remaining: 244ms
987:	learn: 0.2769228	total: 18.5s	remaining: 225ms
988:	learn: 0.2768567	total: 18.5s	remaining: 206ms
989:	learn: 

In [88]:
from sklearn.ensemble import VotingClassifier
model5 = VotingClassifier(estimators=[('lgbm', gs),('dt', dt_cls),('rf', RF_model)], voting='soft')
model5.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_5=model5.predict(X_test)

Model Complete


In [53]:
from sklearn.ensemble import VotingClassifier
model4 = VotingClassifier(estimators=[('Cb', CatBoost),('lgbm', gs),('dt', dt_cls),('rf', RF_model)], voting='soft')
model4.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_4=model4.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 15.8ms	remaining: 15.8s
1:	learn: 0.6679542	total: 32.3ms	remaining: 16.1s
2:	learn: 0.6586634	total: 44.4ms	remaining: 14.8s
3:	learn: 0.6506448	total: 58.4ms	remaining: 14.5s
4:	learn: 0.6408058	total: 76.1ms	remaining: 15.1s
5:	learn: 0.6340672	total: 98ms	remaining: 16.2s
6:	learn: 0.6286264	total: 114ms	remaining: 16.2s
7:	learn: 0.6226329	total: 136ms	remaining: 16.9s
8:	learn: 0.6184221	total: 151ms	remaining: 16.6s
9:	learn: 0.6140516	total: 167ms	remaining: 16.5s
10:	learn: 0.6095748	total: 183ms	remaining: 16.5s
11:	learn: 0.6056595	total: 203ms	remaining: 16.7s
12:	learn: 0.6009331	total: 218ms	remaining: 16.6s
13:	learn: 0.5953545	total: 235ms	remaining: 16.6s
14:	learn: 0.5916079	total: 251ms	remaining: 16.5s
15:	learn: 0.5871947	total: 275ms	remaining: 16.9s
16:	learn: 0.5799814	total: 292ms	remaining: 16.9s
17:	learn: 0.5771092	total: 311ms	remaining: 17s
18:	learn: 0.5719313	total: 330ms	remaining: 17s
19:	learn: 0

166:	learn: 0.4203855	total: 3.33s	remaining: 16.6s
167:	learn: 0.4200934	total: 3.34s	remaining: 16.6s
168:	learn: 0.4197580	total: 3.36s	remaining: 16.5s
169:	learn: 0.4189651	total: 3.37s	remaining: 16.5s
170:	learn: 0.4186179	total: 3.39s	remaining: 16.4s
171:	learn: 0.4181877	total: 3.41s	remaining: 16.4s
172:	learn: 0.4178532	total: 3.43s	remaining: 16.4s
173:	learn: 0.4176437	total: 3.44s	remaining: 16.4s
174:	learn: 0.4173844	total: 3.46s	remaining: 16.3s
175:	learn: 0.4170745	total: 3.48s	remaining: 16.3s
176:	learn: 0.4167306	total: 3.5s	remaining: 16.3s
177:	learn: 0.4163079	total: 3.53s	remaining: 16.3s
178:	learn: 0.4158617	total: 3.54s	remaining: 16.2s
179:	learn: 0.4154727	total: 3.56s	remaining: 16.2s
180:	learn: 0.4152674	total: 3.58s	remaining: 16.2s
181:	learn: 0.4148733	total: 3.59s	remaining: 16.1s
182:	learn: 0.4143350	total: 3.61s	remaining: 16.1s
183:	learn: 0.4140410	total: 3.63s	remaining: 16.1s
184:	learn: 0.4135355	total: 3.64s	remaining: 16s
185:	learn: 0.4

329:	learn: 0.3582579	total: 6.26s	remaining: 12.7s
330:	learn: 0.3578227	total: 6.28s	remaining: 12.7s
331:	learn: 0.3574625	total: 6.3s	remaining: 12.7s
332:	learn: 0.3572630	total: 6.32s	remaining: 12.7s
333:	learn: 0.3571139	total: 6.34s	remaining: 12.6s
334:	learn: 0.3569167	total: 6.36s	remaining: 12.6s
335:	learn: 0.3565053	total: 6.38s	remaining: 12.6s
336:	learn: 0.3563696	total: 6.4s	remaining: 12.6s
337:	learn: 0.3560269	total: 6.42s	remaining: 12.6s
338:	learn: 0.3556117	total: 6.44s	remaining: 12.6s
339:	learn: 0.3553019	total: 6.46s	remaining: 12.5s
340:	learn: 0.3551200	total: 6.47s	remaining: 12.5s
341:	learn: 0.3549500	total: 6.5s	remaining: 12.5s
342:	learn: 0.3545114	total: 6.51s	remaining: 12.5s
343:	learn: 0.3542186	total: 6.54s	remaining: 12.5s
344:	learn: 0.3539000	total: 6.55s	remaining: 12.4s
345:	learn: 0.3535411	total: 6.57s	remaining: 12.4s
346:	learn: 0.3534507	total: 6.59s	remaining: 12.4s
347:	learn: 0.3530517	total: 6.61s	remaining: 12.4s
348:	learn: 0.3

491:	learn: 0.3254191	total: 9.22s	remaining: 9.52s
492:	learn: 0.3252503	total: 9.24s	remaining: 9.5s
493:	learn: 0.3251172	total: 9.26s	remaining: 9.48s
494:	learn: 0.3250046	total: 9.28s	remaining: 9.46s
495:	learn: 0.3249080	total: 9.29s	remaining: 9.44s
496:	learn: 0.3247105	total: 9.31s	remaining: 9.42s
497:	learn: 0.3245770	total: 9.33s	remaining: 9.4s
498:	learn: 0.3244784	total: 9.35s	remaining: 9.38s
499:	learn: 0.3243497	total: 9.36s	remaining: 9.36s
500:	learn: 0.3242095	total: 9.38s	remaining: 9.34s
501:	learn: 0.3240897	total: 9.39s	remaining: 9.32s
502:	learn: 0.3239015	total: 9.41s	remaining: 9.3s
503:	learn: 0.3237987	total: 9.43s	remaining: 9.28s
504:	learn: 0.3236880	total: 9.45s	remaining: 9.26s
505:	learn: 0.3235400	total: 9.47s	remaining: 9.25s
506:	learn: 0.3234257	total: 9.49s	remaining: 9.23s
507:	learn: 0.3233202	total: 9.51s	remaining: 9.21s
508:	learn: 0.3231908	total: 9.53s	remaining: 9.19s
509:	learn: 0.3230773	total: 9.54s	remaining: 9.17s
510:	learn: 0.3

650:	learn: 0.3059759	total: 12.1s	remaining: 6.5s
651:	learn: 0.3058707	total: 12.1s	remaining: 6.48s
652:	learn: 0.3057909	total: 12.2s	remaining: 6.46s
653:	learn: 0.3056714	total: 12.2s	remaining: 6.44s
654:	learn: 0.3055410	total: 12.2s	remaining: 6.42s
655:	learn: 0.3054457	total: 12.2s	remaining: 6.4s
656:	learn: 0.3053621	total: 12.2s	remaining: 6.39s
657:	learn: 0.3052218	total: 12.3s	remaining: 6.37s
658:	learn: 0.3051549	total: 12.3s	remaining: 6.35s
659:	learn: 0.3050008	total: 12.3s	remaining: 6.33s
660:	learn: 0.3048823	total: 12.3s	remaining: 6.32s
661:	learn: 0.3048124	total: 12.3s	remaining: 6.3s
662:	learn: 0.3046744	total: 12.4s	remaining: 6.28s
663:	learn: 0.3046230	total: 12.4s	remaining: 6.27s
664:	learn: 0.3045188	total: 12.4s	remaining: 6.25s
665:	learn: 0.3044218	total: 12.4s	remaining: 6.23s
666:	learn: 0.3043253	total: 12.4s	remaining: 6.21s
667:	learn: 0.3042170	total: 12.5s	remaining: 6.19s
668:	learn: 0.3041603	total: 12.5s	remaining: 6.17s
669:	learn: 0.3

819:	learn: 0.2902293	total: 15.2s	remaining: 3.34s
820:	learn: 0.2901360	total: 15.2s	remaining: 3.32s
821:	learn: 0.2900767	total: 15.3s	remaining: 3.3s
822:	learn: 0.2899938	total: 15.3s	remaining: 3.29s
823:	learn: 0.2899065	total: 15.3s	remaining: 3.27s
824:	learn: 0.2898437	total: 15.3s	remaining: 3.25s
825:	learn: 0.2897834	total: 15.3s	remaining: 3.23s
826:	learn: 0.2897266	total: 15.3s	remaining: 3.21s
827:	learn: 0.2895873	total: 15.4s	remaining: 3.19s
828:	learn: 0.2895197	total: 15.4s	remaining: 3.17s
829:	learn: 0.2893973	total: 15.4s	remaining: 3.15s
830:	learn: 0.2891751	total: 15.4s	remaining: 3.13s
831:	learn: 0.2891185	total: 15.4s	remaining: 3.12s
832:	learn: 0.2890525	total: 15.4s	remaining: 3.1s
833:	learn: 0.2889932	total: 15.5s	remaining: 3.08s
834:	learn: 0.2889123	total: 15.5s	remaining: 3.06s
835:	learn: 0.2888712	total: 15.5s	remaining: 3.04s
836:	learn: 0.2888259	total: 15.5s	remaining: 3.02s
837:	learn: 0.2887533	total: 15.5s	remaining: 3s
838:	learn: 0.288

988:	learn: 0.2768567	total: 18.1s	remaining: 201ms
989:	learn: 0.2768206	total: 18.1s	remaining: 183ms
990:	learn: 0.2767359	total: 18.1s	remaining: 165ms
991:	learn: 0.2766794	total: 18.2s	remaining: 146ms
992:	learn: 0.2766293	total: 18.2s	remaining: 128ms
993:	learn: 0.2765418	total: 18.2s	remaining: 110ms
994:	learn: 0.2764595	total: 18.2s	remaining: 91.5ms
995:	learn: 0.2763725	total: 18.2s	remaining: 73.2ms
996:	learn: 0.2763087	total: 18.2s	remaining: 54.9ms
997:	learn: 0.2762327	total: 18.3s	remaining: 36.6ms
998:	learn: 0.2761429	total: 18.3s	remaining: 18.3ms
999:	learn: 0.2760758	total: 18.3s	remaining: 0us
Model Complete


In [54]:
from sklearn.ensemble import VotingClassifier
model3 = VotingClassifier(estimators=[('Cb', CatBoost),('lgbm', gs),('dt', dt_cls)], voting='soft')
model3.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_3=model3.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 11ms	remaining: 11s
1:	learn: 0.6679542	total: 23.3ms	remaining: 11.6s
2:	learn: 0.6586634	total: 34.7ms	remaining: 11.5s
3:	learn: 0.6506448	total: 47.8ms	remaining: 11.9s
4:	learn: 0.6408058	total: 66.2ms	remaining: 13.2s
5:	learn: 0.6340672	total: 81.2ms	remaining: 13.4s
6:	learn: 0.6286264	total: 100ms	remaining: 14.2s
7:	learn: 0.6226329	total: 114ms	remaining: 14.2s
8:	learn: 0.6184221	total: 138ms	remaining: 15.1s
9:	learn: 0.6140516	total: 153ms	remaining: 15.2s
10:	learn: 0.6095748	total: 177ms	remaining: 15.9s
11:	learn: 0.6056595	total: 196ms	remaining: 16.2s
12:	learn: 0.6009331	total: 218ms	remaining: 16.6s
13:	learn: 0.5953545	total: 234ms	remaining: 16.5s
14:	learn: 0.5916079	total: 258ms	remaining: 16.9s
15:	learn: 0.5871947	total: 275ms	remaining: 16.9s
16:	learn: 0.5799814	total: 302ms	remaining: 17.5s
17:	learn: 0.5771092	total: 318ms	remaining: 17.3s
18:	learn: 0.5719313	total: 336ms	remaining: 17.4s
19:	learn:

160:	learn: 0.4224735	total: 2.99s	remaining: 15.6s
161:	learn: 0.4218811	total: 3.01s	remaining: 15.6s
162:	learn: 0.4215547	total: 3.03s	remaining: 15.6s
163:	learn: 0.4211132	total: 3.05s	remaining: 15.6s
164:	learn: 0.4209344	total: 3.07s	remaining: 15.5s
165:	learn: 0.4207196	total: 3.1s	remaining: 15.6s
166:	learn: 0.4203855	total: 3.11s	remaining: 15.5s
167:	learn: 0.4200934	total: 3.13s	remaining: 15.5s
168:	learn: 0.4197580	total: 3.15s	remaining: 15.5s
169:	learn: 0.4189651	total: 3.17s	remaining: 15.5s
170:	learn: 0.4186179	total: 3.19s	remaining: 15.5s
171:	learn: 0.4181877	total: 3.21s	remaining: 15.4s
172:	learn: 0.4178532	total: 3.23s	remaining: 15.5s
173:	learn: 0.4176437	total: 3.25s	remaining: 15.4s
174:	learn: 0.4173844	total: 3.27s	remaining: 15.4s
175:	learn: 0.4170745	total: 3.29s	remaining: 15.4s
176:	learn: 0.4167306	total: 3.31s	remaining: 15.4s
177:	learn: 0.4163079	total: 3.32s	remaining: 15.4s
178:	learn: 0.4158617	total: 3.34s	remaining: 15.3s
179:	learn: 0

319:	learn: 0.3613531	total: 6.75s	remaining: 14.3s
320:	learn: 0.3611573	total: 6.78s	remaining: 14.3s
321:	learn: 0.3608421	total: 6.81s	remaining: 14.3s
322:	learn: 0.3602778	total: 6.84s	remaining: 14.3s
323:	learn: 0.3600014	total: 6.87s	remaining: 14.3s
324:	learn: 0.3598087	total: 6.9s	remaining: 14.3s
325:	learn: 0.3596011	total: 6.92s	remaining: 14.3s
326:	learn: 0.3591945	total: 6.95s	remaining: 14.3s
327:	learn: 0.3589449	total: 6.97s	remaining: 14.3s
328:	learn: 0.3587682	total: 6.99s	remaining: 14.3s
329:	learn: 0.3582579	total: 7.01s	remaining: 14.2s
330:	learn: 0.3578227	total: 7.03s	remaining: 14.2s
331:	learn: 0.3574625	total: 7.05s	remaining: 14.2s
332:	learn: 0.3572630	total: 7.08s	remaining: 14.2s
333:	learn: 0.3571139	total: 7.09s	remaining: 14.1s
334:	learn: 0.3569167	total: 7.13s	remaining: 14.2s
335:	learn: 0.3565053	total: 7.16s	remaining: 14.1s
336:	learn: 0.3563696	total: 7.18s	remaining: 14.1s
337:	learn: 0.3560269	total: 7.22s	remaining: 14.1s
338:	learn: 0

481:	learn: 0.3268897	total: 10.8s	remaining: 11.6s
482:	learn: 0.3267729	total: 10.8s	remaining: 11.6s
483:	learn: 0.3266848	total: 10.9s	remaining: 11.6s
484:	learn: 0.3265671	total: 10.9s	remaining: 11.6s
485:	learn: 0.3264263	total: 10.9s	remaining: 11.5s
486:	learn: 0.3261930	total: 10.9s	remaining: 11.5s
487:	learn: 0.3260570	total: 10.9s	remaining: 11.5s
488:	learn: 0.3257571	total: 11s	remaining: 11.5s
489:	learn: 0.3256668	total: 11s	remaining: 11.4s
490:	learn: 0.3255310	total: 11s	remaining: 11.4s
491:	learn: 0.3254191	total: 11s	remaining: 11.4s
492:	learn: 0.3252503	total: 11.1s	remaining: 11.4s
493:	learn: 0.3251172	total: 11.1s	remaining: 11.3s
494:	learn: 0.3250046	total: 11.1s	remaining: 11.3s
495:	learn: 0.3249080	total: 11.1s	remaining: 11.3s
496:	learn: 0.3247105	total: 11.2s	remaining: 11.3s
497:	learn: 0.3245770	total: 11.2s	remaining: 11.3s
498:	learn: 0.3244784	total: 11.2s	remaining: 11.3s
499:	learn: 0.3243497	total: 11.2s	remaining: 11.2s
500:	learn: 0.324209

640:	learn: 0.3071205	total: 14.6s	remaining: 8.16s
641:	learn: 0.3069253	total: 14.6s	remaining: 8.14s
642:	learn: 0.3068561	total: 14.6s	remaining: 8.12s
643:	learn: 0.3067364	total: 14.7s	remaining: 8.1s
644:	learn: 0.3066563	total: 14.7s	remaining: 8.07s
645:	learn: 0.3064735	total: 14.7s	remaining: 8.05s
646:	learn: 0.3064081	total: 14.7s	remaining: 8.03s
647:	learn: 0.3062515	total: 14.7s	remaining: 8.01s
648:	learn: 0.3061605	total: 14.8s	remaining: 7.99s
649:	learn: 0.3060862	total: 14.8s	remaining: 7.96s
650:	learn: 0.3059759	total: 14.8s	remaining: 7.94s
651:	learn: 0.3058707	total: 14.8s	remaining: 7.92s
652:	learn: 0.3057909	total: 14.9s	remaining: 7.89s
653:	learn: 0.3056714	total: 14.9s	remaining: 7.87s
654:	learn: 0.3055410	total: 14.9s	remaining: 7.86s
655:	learn: 0.3054457	total: 14.9s	remaining: 7.83s
656:	learn: 0.3053621	total: 15s	remaining: 7.82s
657:	learn: 0.3052218	total: 15s	remaining: 7.79s
658:	learn: 0.3051549	total: 15s	remaining: 7.77s
659:	learn: 0.30500

804:	learn: 0.2915106	total: 19s	remaining: 4.59s
805:	learn: 0.2914168	total: 19s	remaining: 4.57s
806:	learn: 0.2913786	total: 19s	remaining: 4.55s
807:	learn: 0.2913337	total: 19s	remaining: 4.52s
808:	learn: 0.2912794	total: 19.1s	remaining: 4.5s
809:	learn: 0.2911944	total: 19.1s	remaining: 4.47s
810:	learn: 0.2911114	total: 19.1s	remaining: 4.45s
811:	learn: 0.2910277	total: 19.1s	remaining: 4.42s
812:	learn: 0.2909515	total: 19.1s	remaining: 4.4s
813:	learn: 0.2908974	total: 19.1s	remaining: 4.38s
814:	learn: 0.2907856	total: 19.2s	remaining: 4.35s
815:	learn: 0.2907099	total: 19.2s	remaining: 4.33s
816:	learn: 0.2905168	total: 19.2s	remaining: 4.3s
817:	learn: 0.2903982	total: 19.2s	remaining: 4.28s
818:	learn: 0.2902975	total: 19.3s	remaining: 4.25s
819:	learn: 0.2902293	total: 19.3s	remaining: 4.23s
820:	learn: 0.2901360	total: 19.3s	remaining: 4.21s
821:	learn: 0.2900767	total: 19.3s	remaining: 4.18s
822:	learn: 0.2899938	total: 19.3s	remaining: 4.16s
823:	learn: 0.2899065	t

970:	learn: 0.2782399	total: 22.7s	remaining: 678ms
971:	learn: 0.2781542	total: 22.7s	remaining: 655ms
972:	learn: 0.2780641	total: 22.7s	remaining: 631ms
973:	learn: 0.2779606	total: 22.8s	remaining: 608ms
974:	learn: 0.2778747	total: 22.8s	remaining: 584ms
975:	learn: 0.2778203	total: 22.8s	remaining: 560ms
976:	learn: 0.2777686	total: 22.8s	remaining: 537ms
977:	learn: 0.2776804	total: 22.8s	remaining: 514ms
978:	learn: 0.2775982	total: 22.9s	remaining: 491ms
979:	learn: 0.2774787	total: 22.9s	remaining: 467ms
980:	learn: 0.2774271	total: 22.9s	remaining: 444ms
981:	learn: 0.2773683	total: 22.9s	remaining: 420ms
982:	learn: 0.2772605	total: 23s	remaining: 397ms
983:	learn: 0.2771929	total: 23s	remaining: 374ms
984:	learn: 0.2771370	total: 23s	remaining: 350ms
985:	learn: 0.2770458	total: 23s	remaining: 327ms
986:	learn: 0.2769699	total: 23s	remaining: 304ms
987:	learn: 0.2769228	total: 23.1s	remaining: 280ms
988:	learn: 0.2768567	total: 23.1s	remaining: 257ms
989:	learn: 0.2768206	

In [55]:
from sklearn.ensemble import VotingClassifier
model2 = VotingClassifier(estimators=[('Cb', CatBoost),('lgbm', gs)], voting='soft')
model2.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_2=model2.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 12.7ms	remaining: 12.7s
1:	learn: 0.6679542	total: 22.9ms	remaining: 11.4s
2:	learn: 0.6586634	total: 34.3ms	remaining: 11.4s
3:	learn: 0.6506448	total: 48.7ms	remaining: 12.1s
4:	learn: 0.6408058	total: 67ms	remaining: 13.3s
5:	learn: 0.6340672	total: 82.7ms	remaining: 13.7s
6:	learn: 0.6286264	total: 98ms	remaining: 13.9s
7:	learn: 0.6226329	total: 117ms	remaining: 14.6s
8:	learn: 0.6184221	total: 133ms	remaining: 14.6s
9:	learn: 0.6140516	total: 146ms	remaining: 14.5s
10:	learn: 0.6095748	total: 161ms	remaining: 14.4s
11:	learn: 0.6056595	total: 178ms	remaining: 14.6s
12:	learn: 0.6009331	total: 197ms	remaining: 14.9s
13:	learn: 0.5953545	total: 222ms	remaining: 15.7s
14:	learn: 0.5916079	total: 239ms	remaining: 15.7s
15:	learn: 0.5871947	total: 257ms	remaining: 15.8s
16:	learn: 0.5799814	total: 273ms	remaining: 15.8s
17:	learn: 0.5771092	total: 290ms	remaining: 15.8s
18:	learn: 0.5719313	total: 306ms	remaining: 15.8s
19:	learn

160:	learn: 0.4224735	total: 3s	remaining: 15.6s
161:	learn: 0.4218811	total: 3.01s	remaining: 15.6s
162:	learn: 0.4215547	total: 3.03s	remaining: 15.6s
163:	learn: 0.4211132	total: 3.05s	remaining: 15.6s
164:	learn: 0.4209344	total: 3.07s	remaining: 15.5s
165:	learn: 0.4207196	total: 3.09s	remaining: 15.5s
166:	learn: 0.4203855	total: 3.1s	remaining: 15.5s
167:	learn: 0.4200934	total: 3.13s	remaining: 15.5s
168:	learn: 0.4197580	total: 3.14s	remaining: 15.4s
169:	learn: 0.4189651	total: 3.16s	remaining: 15.4s
170:	learn: 0.4186179	total: 3.18s	remaining: 15.4s
171:	learn: 0.4181877	total: 3.21s	remaining: 15.4s
172:	learn: 0.4178532	total: 3.22s	remaining: 15.4s
173:	learn: 0.4176437	total: 3.24s	remaining: 15.4s
174:	learn: 0.4173844	total: 3.26s	remaining: 15.4s
175:	learn: 0.4170745	total: 3.27s	remaining: 15.3s
176:	learn: 0.4167306	total: 3.29s	remaining: 15.3s
177:	learn: 0.4163079	total: 3.31s	remaining: 15.3s
178:	learn: 0.4158617	total: 3.33s	remaining: 15.3s
179:	learn: 0.41

324:	learn: 0.3598087	total: 6.2s	remaining: 12.9s
325:	learn: 0.3596011	total: 6.22s	remaining: 12.9s
326:	learn: 0.3591945	total: 6.24s	remaining: 12.8s
327:	learn: 0.3589449	total: 6.25s	remaining: 12.8s
328:	learn: 0.3587682	total: 6.27s	remaining: 12.8s
329:	learn: 0.3582579	total: 6.29s	remaining: 12.8s
330:	learn: 0.3578227	total: 6.31s	remaining: 12.8s
331:	learn: 0.3574625	total: 6.33s	remaining: 12.7s
332:	learn: 0.3572630	total: 6.35s	remaining: 12.7s
333:	learn: 0.3571139	total: 6.37s	remaining: 12.7s
334:	learn: 0.3569167	total: 6.38s	remaining: 12.7s
335:	learn: 0.3565053	total: 6.4s	remaining: 12.7s
336:	learn: 0.3563696	total: 6.42s	remaining: 12.6s
337:	learn: 0.3560269	total: 6.44s	remaining: 12.6s
338:	learn: 0.3556117	total: 6.46s	remaining: 12.6s
339:	learn: 0.3553019	total: 6.47s	remaining: 12.6s
340:	learn: 0.3551200	total: 6.49s	remaining: 12.5s
341:	learn: 0.3549500	total: 6.51s	remaining: 12.5s
342:	learn: 0.3545114	total: 6.53s	remaining: 12.5s
343:	learn: 0.

487:	learn: 0.3260570	total: 9.21s	remaining: 9.66s
488:	learn: 0.3257571	total: 9.23s	remaining: 9.64s
489:	learn: 0.3256668	total: 9.24s	remaining: 9.62s
490:	learn: 0.3255310	total: 9.26s	remaining: 9.6s
491:	learn: 0.3254191	total: 9.28s	remaining: 9.58s
492:	learn: 0.3252503	total: 9.3s	remaining: 9.56s
493:	learn: 0.3251172	total: 9.32s	remaining: 9.54s
494:	learn: 0.3250046	total: 9.34s	remaining: 9.53s
495:	learn: 0.3249080	total: 9.36s	remaining: 9.51s
496:	learn: 0.3247105	total: 9.38s	remaining: 9.49s
497:	learn: 0.3245770	total: 9.4s	remaining: 9.47s
498:	learn: 0.3244784	total: 9.42s	remaining: 9.45s
499:	learn: 0.3243497	total: 9.44s	remaining: 9.44s
500:	learn: 0.3242095	total: 9.46s	remaining: 9.42s
501:	learn: 0.3240897	total: 9.49s	remaining: 9.41s
502:	learn: 0.3239015	total: 9.5s	remaining: 9.39s
503:	learn: 0.3237987	total: 9.53s	remaining: 9.38s
504:	learn: 0.3236880	total: 9.56s	remaining: 9.37s
505:	learn: 0.3235400	total: 9.58s	remaining: 9.35s
506:	learn: 0.32

649:	learn: 0.3060862	total: 12.7s	remaining: 6.86s
650:	learn: 0.3059759	total: 12.8s	remaining: 6.84s
651:	learn: 0.3058707	total: 12.8s	remaining: 6.82s
652:	learn: 0.3057909	total: 12.8s	remaining: 6.8s
653:	learn: 0.3056714	total: 12.8s	remaining: 6.78s
654:	learn: 0.3055410	total: 12.8s	remaining: 6.76s
655:	learn: 0.3054457	total: 12.9s	remaining: 6.74s
656:	learn: 0.3053621	total: 12.9s	remaining: 6.72s
657:	learn: 0.3052218	total: 12.9s	remaining: 6.71s
658:	learn: 0.3051549	total: 12.9s	remaining: 6.69s
659:	learn: 0.3050008	total: 12.9s	remaining: 6.67s
660:	learn: 0.3048823	total: 13s	remaining: 6.65s
661:	learn: 0.3048124	total: 13s	remaining: 6.63s
662:	learn: 0.3046744	total: 13s	remaining: 6.61s
663:	learn: 0.3046230	total: 13s	remaining: 6.58s
664:	learn: 0.3045188	total: 13s	remaining: 6.56s
665:	learn: 0.3044218	total: 13s	remaining: 6.54s
666:	learn: 0.3043253	total: 13.1s	remaining: 6.53s
667:	learn: 0.3042170	total: 13.1s	remaining: 6.51s
668:	learn: 0.3041603	tot

810:	learn: 0.2911114	total: 16s	remaining: 3.72s
811:	learn: 0.2910277	total: 16s	remaining: 3.7s
812:	learn: 0.2909515	total: 16s	remaining: 3.68s
813:	learn: 0.2908974	total: 16s	remaining: 3.66s
814:	learn: 0.2907856	total: 16s	remaining: 3.64s
815:	learn: 0.2907099	total: 16.1s	remaining: 3.62s
816:	learn: 0.2905168	total: 16.1s	remaining: 3.6s
817:	learn: 0.2903982	total: 16.1s	remaining: 3.58s
818:	learn: 0.2902975	total: 16.1s	remaining: 3.56s
819:	learn: 0.2902293	total: 16.1s	remaining: 3.54s
820:	learn: 0.2901360	total: 16.2s	remaining: 3.52s
821:	learn: 0.2900767	total: 16.2s	remaining: 3.5s
822:	learn: 0.2899938	total: 16.2s	remaining: 3.48s
823:	learn: 0.2899065	total: 16.2s	remaining: 3.46s
824:	learn: 0.2898437	total: 16.2s	remaining: 3.44s
825:	learn: 0.2897834	total: 16.3s	remaining: 3.42s
826:	learn: 0.2897266	total: 16.3s	remaining: 3.4s
827:	learn: 0.2895873	total: 16.3s	remaining: 3.38s
828:	learn: 0.2895197	total: 16.3s	remaining: 3.37s
829:	learn: 0.2893973	tota

969:	learn: 0.2783303	total: 19.7s	remaining: 608ms
970:	learn: 0.2782399	total: 19.7s	remaining: 587ms
971:	learn: 0.2781542	total: 19.7s	remaining: 567ms
972:	learn: 0.2780641	total: 19.7s	remaining: 547ms
973:	learn: 0.2779606	total: 19.7s	remaining: 527ms
974:	learn: 0.2778747	total: 19.8s	remaining: 507ms
975:	learn: 0.2778203	total: 19.8s	remaining: 486ms
976:	learn: 0.2777686	total: 19.8s	remaining: 467ms
977:	learn: 0.2776804	total: 19.8s	remaining: 446ms
978:	learn: 0.2775982	total: 19.9s	remaining: 426ms
979:	learn: 0.2774787	total: 19.9s	remaining: 406ms
980:	learn: 0.2774271	total: 19.9s	remaining: 386ms
981:	learn: 0.2773683	total: 19.9s	remaining: 365ms
982:	learn: 0.2772605	total: 20s	remaining: 345ms
983:	learn: 0.2771929	total: 20s	remaining: 325ms
984:	learn: 0.2771370	total: 20s	remaining: 305ms
985:	learn: 0.2770458	total: 20s	remaining: 285ms
986:	learn: 0.2769699	total: 20.1s	remaining: 264ms
987:	learn: 0.2769228	total: 20.1s	remaining: 244ms
988:	learn: 0.276856

In [56]:
from sklearn.ensemble import VotingClassifier
model1 = VotingClassifier(estimators=[('Cb', CatBoost),('rf', RF_model),('lgbm', gs)], voting='soft')
model1.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred_1=model1.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 17.2ms	remaining: 17.2s
1:	learn: 0.6679542	total: 34.5ms	remaining: 17.2s
2:	learn: 0.6586634	total: 60.9ms	remaining: 20.2s
3:	learn: 0.6506448	total: 78.6ms	remaining: 19.6s
4:	learn: 0.6408058	total: 105ms	remaining: 20.9s
5:	learn: 0.6340672	total: 123ms	remaining: 20.4s
6:	learn: 0.6286264	total: 157ms	remaining: 22.2s
7:	learn: 0.6226329	total: 177ms	remaining: 21.9s
8:	learn: 0.6184221	total: 194ms	remaining: 21.4s
9:	learn: 0.6140516	total: 209ms	remaining: 20.7s
10:	learn: 0.6095748	total: 228ms	remaining: 20.5s
11:	learn: 0.6056595	total: 247ms	remaining: 20.4s
12:	learn: 0.6009331	total: 269ms	remaining: 20.4s
13:	learn: 0.5953545	total: 295ms	remaining: 20.8s
14:	learn: 0.5916079	total: 314ms	remaining: 20.6s
15:	learn: 0.5871947	total: 332ms	remaining: 20.4s
16:	learn: 0.5799814	total: 365ms	remaining: 21.1s
17:	learn: 0.5771092	total: 383ms	remaining: 20.9s
18:	learn: 0.5719313	total: 405ms	remaining: 20.9s
19:	lear

161:	learn: 0.4218811	total: 3.42s	remaining: 17.7s
162:	learn: 0.4215547	total: 3.44s	remaining: 17.7s
163:	learn: 0.4211132	total: 3.46s	remaining: 17.6s
164:	learn: 0.4209344	total: 3.48s	remaining: 17.6s
165:	learn: 0.4207196	total: 3.51s	remaining: 17.6s
166:	learn: 0.4203855	total: 3.53s	remaining: 17.6s
167:	learn: 0.4200934	total: 3.55s	remaining: 17.6s
168:	learn: 0.4197580	total: 3.59s	remaining: 17.7s
169:	learn: 0.4189651	total: 3.62s	remaining: 17.6s
170:	learn: 0.4186179	total: 3.64s	remaining: 17.6s
171:	learn: 0.4181877	total: 3.66s	remaining: 17.6s
172:	learn: 0.4178532	total: 3.68s	remaining: 17.6s
173:	learn: 0.4176437	total: 3.71s	remaining: 17.6s
174:	learn: 0.4173844	total: 3.72s	remaining: 17.6s
175:	learn: 0.4170745	total: 3.74s	remaining: 17.5s
176:	learn: 0.4167306	total: 3.76s	remaining: 17.5s
177:	learn: 0.4163079	total: 3.78s	remaining: 17.5s
178:	learn: 0.4158617	total: 3.8s	remaining: 17.4s
179:	learn: 0.4154727	total: 3.82s	remaining: 17.4s
180:	learn: 0

323:	learn: 0.3600014	total: 7.07s	remaining: 14.7s
324:	learn: 0.3598087	total: 7.09s	remaining: 14.7s
325:	learn: 0.3596011	total: 7.11s	remaining: 14.7s
326:	learn: 0.3591945	total: 7.13s	remaining: 14.7s
327:	learn: 0.3589449	total: 7.14s	remaining: 14.6s
328:	learn: 0.3587682	total: 7.16s	remaining: 14.6s
329:	learn: 0.3582579	total: 7.18s	remaining: 14.6s
330:	learn: 0.3578227	total: 7.2s	remaining: 14.6s
331:	learn: 0.3574625	total: 7.22s	remaining: 14.5s
332:	learn: 0.3572630	total: 7.24s	remaining: 14.5s
333:	learn: 0.3571139	total: 7.26s	remaining: 14.5s
334:	learn: 0.3569167	total: 7.28s	remaining: 14.4s
335:	learn: 0.3565053	total: 7.29s	remaining: 14.4s
336:	learn: 0.3563696	total: 7.32s	remaining: 14.4s
337:	learn: 0.3560269	total: 7.34s	remaining: 14.4s
338:	learn: 0.3556117	total: 7.36s	remaining: 14.3s
339:	learn: 0.3553019	total: 7.37s	remaining: 14.3s
340:	learn: 0.3551200	total: 7.39s	remaining: 14.3s
341:	learn: 0.3549500	total: 7.41s	remaining: 14.3s
342:	learn: 0

482:	learn: 0.3267729	total: 10.8s	remaining: 11.5s
483:	learn: 0.3266848	total: 10.8s	remaining: 11.5s
484:	learn: 0.3265671	total: 10.8s	remaining: 11.5s
485:	learn: 0.3264263	total: 10.8s	remaining: 11.5s
486:	learn: 0.3261930	total: 10.9s	remaining: 11.4s
487:	learn: 0.3260570	total: 10.9s	remaining: 11.4s
488:	learn: 0.3257571	total: 10.9s	remaining: 11.4s
489:	learn: 0.3256668	total: 11s	remaining: 11.4s
490:	learn: 0.3255310	total: 11s	remaining: 11.4s
491:	learn: 0.3254191	total: 11s	remaining: 11.4s
492:	learn: 0.3252503	total: 11.1s	remaining: 11.4s
493:	learn: 0.3251172	total: 11.1s	remaining: 11.3s
494:	learn: 0.3250046	total: 11.1s	remaining: 11.3s
495:	learn: 0.3249080	total: 11.1s	remaining: 11.3s
496:	learn: 0.3247105	total: 11.1s	remaining: 11.3s
497:	learn: 0.3245770	total: 11.2s	remaining: 11.3s
498:	learn: 0.3244784	total: 11.2s	remaining: 11.2s
499:	learn: 0.3243497	total: 11.2s	remaining: 11.2s
500:	learn: 0.3242095	total: 11.2s	remaining: 11.2s
501:	learn: 0.3240

642:	learn: 0.3068561	total: 14.2s	remaining: 7.87s
643:	learn: 0.3067364	total: 14.2s	remaining: 7.84s
644:	learn: 0.3066563	total: 14.2s	remaining: 7.82s
645:	learn: 0.3064735	total: 14.2s	remaining: 7.8s
646:	learn: 0.3064081	total: 14.3s	remaining: 7.78s
647:	learn: 0.3062515	total: 14.3s	remaining: 7.75s
648:	learn: 0.3061605	total: 14.3s	remaining: 7.73s
649:	learn: 0.3060862	total: 14.3s	remaining: 7.7s
650:	learn: 0.3059759	total: 14.3s	remaining: 7.68s
651:	learn: 0.3058707	total: 14.3s	remaining: 7.66s
652:	learn: 0.3057909	total: 14.4s	remaining: 7.63s
653:	learn: 0.3056714	total: 14.4s	remaining: 7.61s
654:	learn: 0.3055410	total: 14.4s	remaining: 7.59s
655:	learn: 0.3054457	total: 14.4s	remaining: 7.57s
656:	learn: 0.3053621	total: 14.4s	remaining: 7.54s
657:	learn: 0.3052218	total: 14.5s	remaining: 7.52s
658:	learn: 0.3051549	total: 14.5s	remaining: 7.5s
659:	learn: 0.3050008	total: 14.5s	remaining: 7.47s
660:	learn: 0.3048823	total: 14.5s	remaining: 7.45s
661:	learn: 0.3

807:	learn: 0.2913337	total: 17.5s	remaining: 4.15s
808:	learn: 0.2912794	total: 17.5s	remaining: 4.13s
809:	learn: 0.2911944	total: 17.5s	remaining: 4.1s
810:	learn: 0.2911114	total: 17.5s	remaining: 4.08s
811:	learn: 0.2910277	total: 17.5s	remaining: 4.06s
812:	learn: 0.2909515	total: 17.5s	remaining: 4.04s
813:	learn: 0.2908974	total: 17.6s	remaining: 4.01s
814:	learn: 0.2907856	total: 17.6s	remaining: 3.99s
815:	learn: 0.2907099	total: 17.6s	remaining: 3.97s
816:	learn: 0.2905168	total: 17.6s	remaining: 3.95s
817:	learn: 0.2903982	total: 17.7s	remaining: 3.93s
818:	learn: 0.2902975	total: 17.7s	remaining: 3.91s
819:	learn: 0.2902293	total: 17.7s	remaining: 3.89s
820:	learn: 0.2901360	total: 17.7s	remaining: 3.86s
821:	learn: 0.2900767	total: 17.7s	remaining: 3.84s
822:	learn: 0.2899938	total: 17.8s	remaining: 3.82s
823:	learn: 0.2899065	total: 17.8s	remaining: 3.8s
824:	learn: 0.2898437	total: 17.8s	remaining: 3.78s
825:	learn: 0.2897834	total: 17.8s	remaining: 3.75s
826:	learn: 0.

967:	learn: 0.2785060	total: 20.5s	remaining: 677ms
968:	learn: 0.2784270	total: 20.5s	remaining: 656ms
969:	learn: 0.2783303	total: 20.5s	remaining: 635ms
970:	learn: 0.2782399	total: 20.5s	remaining: 613ms
971:	learn: 0.2781542	total: 20.6s	remaining: 592ms
972:	learn: 0.2780641	total: 20.6s	remaining: 571ms
973:	learn: 0.2779606	total: 20.6s	remaining: 550ms
974:	learn: 0.2778747	total: 20.6s	remaining: 528ms
975:	learn: 0.2778203	total: 20.6s	remaining: 507ms
976:	learn: 0.2777686	total: 20.6s	remaining: 486ms
977:	learn: 0.2776804	total: 20.7s	remaining: 465ms
978:	learn: 0.2775982	total: 20.7s	remaining: 444ms
979:	learn: 0.2774787	total: 20.7s	remaining: 422ms
980:	learn: 0.2774271	total: 20.7s	remaining: 401ms
981:	learn: 0.2773683	total: 20.7s	remaining: 380ms
982:	learn: 0.2772605	total: 20.8s	remaining: 359ms
983:	learn: 0.2771929	total: 20.8s	remaining: 338ms
984:	learn: 0.2771370	total: 20.8s	remaining: 317ms
985:	learn: 0.2770458	total: 20.8s	remaining: 295ms
986:	learn: 

In [57]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier(estimators=[('Cb', CatBoost),('rf', RF_model),('lgbm', LightGBM)], voting='soft')
model.fit(sm_xtrain,sm_ytrain)

print('Model Complete')
# In[55]:


Ensemble_pred=model.predict(X_test)

Learning rate set to 0.05013
0:	learn: 0.6817850	total: 16.5ms	remaining: 16.5s
1:	learn: 0.6679542	total: 35.6ms	remaining: 17.8s
2:	learn: 0.6586634	total: 49ms	remaining: 16.3s
3:	learn: 0.6506448	total: 76.3ms	remaining: 19s
4:	learn: 0.6408058	total: 111ms	remaining: 22.2s
5:	learn: 0.6340672	total: 129ms	remaining: 21.4s
6:	learn: 0.6286264	total: 153ms	remaining: 21.6s
7:	learn: 0.6226329	total: 168ms	remaining: 20.9s
8:	learn: 0.6184221	total: 184ms	remaining: 20.3s
9:	learn: 0.6140516	total: 202ms	remaining: 20s
10:	learn: 0.6095748	total: 218ms	remaining: 19.6s
11:	learn: 0.6056595	total: 236ms	remaining: 19.5s
12:	learn: 0.6009331	total: 254ms	remaining: 19.3s
13:	learn: 0.5953545	total: 274ms	remaining: 19.3s
14:	learn: 0.5916079	total: 292ms	remaining: 19.2s
15:	learn: 0.5871947	total: 310ms	remaining: 19s
16:	learn: 0.5799814	total: 326ms	remaining: 18.8s
17:	learn: 0.5771092	total: 343ms	remaining: 18.7s
18:	learn: 0.5719313	total: 366ms	remaining: 18.9s
19:	learn: 0.568

167:	learn: 0.4200934	total: 3.31s	remaining: 16.4s
168:	learn: 0.4197580	total: 3.33s	remaining: 16.4s
169:	learn: 0.4189651	total: 3.35s	remaining: 16.3s
170:	learn: 0.4186179	total: 3.37s	remaining: 16.3s
171:	learn: 0.4181877	total: 3.38s	remaining: 16.3s
172:	learn: 0.4178532	total: 3.4s	remaining: 16.3s
173:	learn: 0.4176437	total: 3.42s	remaining: 16.2s
174:	learn: 0.4173844	total: 3.44s	remaining: 16.2s
175:	learn: 0.4170745	total: 3.45s	remaining: 16.2s
176:	learn: 0.4167306	total: 3.47s	remaining: 16.2s
177:	learn: 0.4163079	total: 3.49s	remaining: 16.1s
178:	learn: 0.4158617	total: 3.51s	remaining: 16.1s
179:	learn: 0.4154727	total: 3.53s	remaining: 16.1s
180:	learn: 0.4152674	total: 3.55s	remaining: 16.1s
181:	learn: 0.4148733	total: 3.57s	remaining: 16s
182:	learn: 0.4143350	total: 3.59s	remaining: 16s
183:	learn: 0.4140410	total: 3.62s	remaining: 16.1s
184:	learn: 0.4135355	total: 3.64s	remaining: 16s
185:	learn: 0.4131688	total: 3.66s	remaining: 16s
186:	learn: 0.4130077

326:	learn: 0.3591945	total: 6.22s	remaining: 12.8s
327:	learn: 0.3589449	total: 6.24s	remaining: 12.8s
328:	learn: 0.3587682	total: 6.25s	remaining: 12.8s
329:	learn: 0.3582579	total: 6.27s	remaining: 12.7s
330:	learn: 0.3578227	total: 6.29s	remaining: 12.7s
331:	learn: 0.3574625	total: 6.31s	remaining: 12.7s
332:	learn: 0.3572630	total: 6.33s	remaining: 12.7s
333:	learn: 0.3571139	total: 6.36s	remaining: 12.7s
334:	learn: 0.3569167	total: 6.37s	remaining: 12.7s
335:	learn: 0.3565053	total: 6.4s	remaining: 12.6s
336:	learn: 0.3563696	total: 6.41s	remaining: 12.6s
337:	learn: 0.3560269	total: 6.43s	remaining: 12.6s
338:	learn: 0.3556117	total: 6.45s	remaining: 12.6s
339:	learn: 0.3553019	total: 6.46s	remaining: 12.6s
340:	learn: 0.3551200	total: 6.49s	remaining: 12.5s
341:	learn: 0.3549500	total: 6.51s	remaining: 12.5s
342:	learn: 0.3545114	total: 6.53s	remaining: 12.5s
343:	learn: 0.3542186	total: 6.54s	remaining: 12.5s
344:	learn: 0.3539000	total: 6.56s	remaining: 12.5s
345:	learn: 0

488:	learn: 0.3257571	total: 9.53s	remaining: 9.95s
489:	learn: 0.3256668	total: 9.54s	remaining: 9.93s
490:	learn: 0.3255310	total: 9.57s	remaining: 9.92s
491:	learn: 0.3254191	total: 9.59s	remaining: 9.9s
492:	learn: 0.3252503	total: 9.6s	remaining: 9.88s
493:	learn: 0.3251172	total: 9.62s	remaining: 9.86s
494:	learn: 0.3250046	total: 9.64s	remaining: 9.83s
495:	learn: 0.3249080	total: 9.66s	remaining: 9.82s
496:	learn: 0.3247105	total: 9.68s	remaining: 9.79s
497:	learn: 0.3245770	total: 9.72s	remaining: 9.79s
498:	learn: 0.3244784	total: 9.74s	remaining: 9.78s
499:	learn: 0.3243497	total: 9.76s	remaining: 9.76s
500:	learn: 0.3242095	total: 9.78s	remaining: 9.74s
501:	learn: 0.3240897	total: 9.79s	remaining: 9.72s
502:	learn: 0.3239015	total: 9.81s	remaining: 9.7s
503:	learn: 0.3237987	total: 9.83s	remaining: 9.68s
504:	learn: 0.3236880	total: 9.85s	remaining: 9.66s
505:	learn: 0.3235400	total: 9.87s	remaining: 9.64s
506:	learn: 0.3234257	total: 9.89s	remaining: 9.62s
507:	learn: 0.3

653:	learn: 0.3056714	total: 13.1s	remaining: 6.91s
654:	learn: 0.3055410	total: 13.1s	remaining: 6.89s
655:	learn: 0.3054457	total: 13.1s	remaining: 6.87s
656:	learn: 0.3053621	total: 13.1s	remaining: 6.85s
657:	learn: 0.3052218	total: 13.1s	remaining: 6.83s
658:	learn: 0.3051549	total: 13.1s	remaining: 6.8s
659:	learn: 0.3050008	total: 13.2s	remaining: 6.78s
660:	learn: 0.3048823	total: 13.2s	remaining: 6.76s
661:	learn: 0.3048124	total: 13.2s	remaining: 6.74s
662:	learn: 0.3046744	total: 13.3s	remaining: 6.74s
663:	learn: 0.3046230	total: 13.3s	remaining: 6.72s
664:	learn: 0.3045188	total: 13.3s	remaining: 6.7s
665:	learn: 0.3044218	total: 13.3s	remaining: 6.68s
666:	learn: 0.3043253	total: 13.3s	remaining: 6.66s
667:	learn: 0.3042170	total: 13.4s	remaining: 6.63s
668:	learn: 0.3041603	total: 13.4s	remaining: 6.62s
669:	learn: 0.3041097	total: 13.4s	remaining: 6.59s
670:	learn: 0.3040226	total: 13.4s	remaining: 6.57s
671:	learn: 0.3039209	total: 13.4s	remaining: 6.55s
672:	learn: 0.

812:	learn: 0.2909515	total: 16.4s	remaining: 3.77s
813:	learn: 0.2908974	total: 16.4s	remaining: 3.75s
814:	learn: 0.2907856	total: 16.4s	remaining: 3.73s
815:	learn: 0.2907099	total: 16.4s	remaining: 3.71s
816:	learn: 0.2905168	total: 16.5s	remaining: 3.69s
817:	learn: 0.2903982	total: 16.5s	remaining: 3.67s
818:	learn: 0.2902975	total: 16.5s	remaining: 3.65s
819:	learn: 0.2902293	total: 16.5s	remaining: 3.63s
820:	learn: 0.2901360	total: 16.5s	remaining: 3.6s
821:	learn: 0.2900767	total: 16.6s	remaining: 3.59s
822:	learn: 0.2899938	total: 16.6s	remaining: 3.57s
823:	learn: 0.2899065	total: 16.6s	remaining: 3.55s
824:	learn: 0.2898437	total: 16.6s	remaining: 3.53s
825:	learn: 0.2897834	total: 16.6s	remaining: 3.51s
826:	learn: 0.2897266	total: 16.7s	remaining: 3.49s
827:	learn: 0.2895873	total: 16.7s	remaining: 3.47s
828:	learn: 0.2895197	total: 16.7s	remaining: 3.45s
829:	learn: 0.2893973	total: 16.8s	remaining: 3.44s
830:	learn: 0.2891751	total: 16.8s	remaining: 3.42s
831:	learn: 0

977:	learn: 0.2776804	total: 20.1s	remaining: 453ms
978:	learn: 0.2775982	total: 20.2s	remaining: 432ms
979:	learn: 0.2774787	total: 20.2s	remaining: 412ms
980:	learn: 0.2774271	total: 20.2s	remaining: 391ms
981:	learn: 0.2773683	total: 20.2s	remaining: 371ms
982:	learn: 0.2772605	total: 20.2s	remaining: 350ms
983:	learn: 0.2771929	total: 20.3s	remaining: 329ms
984:	learn: 0.2771370	total: 20.3s	remaining: 309ms
985:	learn: 0.2770458	total: 20.3s	remaining: 288ms
986:	learn: 0.2769699	total: 20.3s	remaining: 268ms
987:	learn: 0.2769228	total: 20.3s	remaining: 247ms
988:	learn: 0.2768567	total: 20.4s	remaining: 226ms
989:	learn: 0.2768206	total: 20.4s	remaining: 206ms
990:	learn: 0.2767359	total: 20.4s	remaining: 185ms
991:	learn: 0.2766794	total: 20.4s	remaining: 165ms
992:	learn: 0.2766293	total: 20.4s	remaining: 144ms
993:	learn: 0.2765418	total: 20.5s	remaining: 123ms
994:	learn: 0.2764595	total: 20.5s	remaining: 103ms
995:	learn: 0.2763725	total: 20.5s	remaining: 82.5ms
996:	learn:

In [47]:
Conf_metrics_emsem = metrics.confusion_matrix(test_y, Ensemble_pred,labels=[0,1])
print(Conf_metrics_emsem)

[[30  7]
 [ 9  4]]


In [276]:
Conf_metrics_emsem = metrics.confusion_matrix(test_y, Ensemble_pred,labels=[0,1])
print(Conf_metrics_emsem)

[[30  7]
 [ 9  4]]


# VALIDATION

In [89]:
X1=d1.loc[:, d1.columns != 'ID']
y1=d1['Response']
X1.drop(columns=['Response'],inplace=True)
X1=X1[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Reco_Insurance_Type','Is_Spouse']]

C:\Python\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [90]:
X.columns

Index(['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Reco_Insurance_Type'],
      dtype='object')

In [60]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X,y, test_size=0.20, random_state=100)

In [62]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.652 (0.009)


In [63]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model1
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.647 (0.008)


In [64]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model2
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.650 (0.008)


In [65]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model3
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.642 (0.010)


In [66]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model4
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.641 (0.010)


In [67]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model5
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.636 (0.010)


In [68]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model6
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

roc_auc: 0.639 (0.010)


# TEST

In [91]:
Test=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\test_YCcRUnU.csv')

In [92]:
Test.head()

ID City_Code  Region_Code Accomodation_Type Reco_Insurance_Type  \
0  50883        C1          156             Owned          Individual   
1  50884        C4            7             Owned               Joint   
2  50885        C1          564            Rented          Individual   
3  50886        C3         1177            Rented          Individual   
4  50887        C1          951             Owned          Individual   

   Upper_Age  Lower_Age Is_Spouse Health Indicator Holding_Policy_Duration  \
0         30         30        No              NaN                     6.0   
1         69         68       Yes               X1                     3.0   
2         28         28        No               X3                     2.0   
3         23         23        No               X3                     3.0   
4         75         75        No               X3                     NaN   

   Holding_Policy_Type  Reco_Policy_Cat  Reco_Policy_Premium  
0                  3.0                5              11934.0  
1                  3.0               18              32204.8  
2                  4.0               17               9240.0  
3                  3.0               18               9086.0  
4                  NaN                5              22534.0

# DATA Pre_Processing

In [93]:
Test['Holding_Policy_Duration']=Test['Holding_Policy_Duration'].replace('14+',15)

In [94]:
Test=Test.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category'})

In [95]:
Test['Accomodation_Type']=Test['Accomodation_Type'].cat.codes
Test['Reco_Insurance_Type']=Test['Reco_Insurance_Type'].cat.codes
Test['City_Code']=Test['City_Code'].cat.codes

In [96]:
Test['Health Indicator']=Test['Health Indicator'].cat.codes
Test['Health Indicator']=Test['Health Indicator'].replace(-1, np.nan)

In [97]:
Test.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium'],
      dtype='object')

In [98]:
Test=Test[['ID','City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Accomodation_Type','Reco_Insurance_Type']]

In [99]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
Test_data = pd.DataFrame(my_imputer.fit_transform(Test))
Test_data.columns=Test.columns
Test_data.index=Test.index

In [100]:
Test_data[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(Test_data[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

In [101]:
Final_Test_data=Test_data.drop(columns=['ID'])

In [102]:
Final_Test_data=Final_Test_data[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat','Reco_Policy_Premium','Reco_Insurance_Type']]

In [103]:
Final_Test_data.head()

City_Code  Region_Code  Upper_Age  Lower_Age  Reco_Policy_Cat  \
0  -1.231415    -1.107355  -0.862252  -0.738218        -1.608585   
1   1.249651    -1.210948   1.398027   1.462293         0.454036   
2  -1.231415    -0.823692  -0.978164  -0.854035         0.295373   
3   0.588033    -0.397502  -1.267943  -1.143576         0.454036   
4  -1.231415    -0.554629   1.745763   1.867650        -1.608585   

   Reco_Policy_Premium  Reco_Insurance_Type  
0            -0.351856                  0.0  
1             2.767762                  1.0  
2            -0.766455                  0.0  
3            -0.790155                  0.0  
4             1.279454                  0.0

# ModelTesting

In [104]:
y_final=model.predict_proba(Final_Test_data)[:,1]

In [105]:
y_final_1=model1.predict_proba(Final_Test_data)[:,1]

In [106]:
y_final_2=model2.predict_proba(Final_Test_data)[:,1]

In [107]:
y_final_3=model3.predict_proba(Final_Test_data)[:,1]

In [108]:
y_final_4=model4.predict_proba(Final_Test_data)[:,1]

In [109]:
y_final_5=model5.predict_proba(Final_Test_data)[:,1]

In [110]:
y_final_6=model6.predict_proba(Final_Test_data)[:,1]

In [126]:
LightGBM.predict_proba(Final_Test_data)[:,1]

array([0.96803705, 0.9788022 , 0.99066628, ..., 0.00273949, 0.99466306,
       0.96389664])

In [146]:
Test_data['Response']=y_final_6

In [148]:
y_final_6

array([0.87813971, 0.84306   , 0.98260661, ..., 0.01644177, 0.97921859,
       0.91489735])

In [149]:
result=Test_data[['ID','Response']]

In [150]:
result['Response'].value_counts()

0.879124    2
0.070788    2
0.830233    2
0.874486    2
0.962746    2
           ..
0.521848    1
0.990588    1
0.921108    1
0.970268    1
0.899976    1
Name: Response, Length: 21791, dtype: int64

In [ ]:
result.head()

In [151]:
result.to_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\model_6.csv')

In [152]:
y_final

array([0.87813971, 0.84306   , 0.98260661, ..., 0.01644177, 0.97921859,
       0.91489735])

In [153]:
y_final_1

array([0.86609998, 0.84080966, 0.98333558, ..., 0.01888201, 0.97642199,
       0.8691296 ])

In [154]:
y_final_2

array([0.96832454, 0.9292145 , 0.99779449, ..., 0.02732301, 0.99632707,
       0.90566151])

In [155]:
y_final_3

array([0.97888303, 0.95280966, 0.99852966, ..., 0.01821534, 0.99755138,
       0.93710767])

In [156]:
y_final_4

array([0.90360465, 0.88360725, 0.98769114, ..., 0.0156615 , 0.98237756,
       0.9054145 ])

In [157]:
y_final_5

array([0.88068778, 0.891083  , 0.9822395 , ..., 0.00577357, 0.98557954,
       0.87431942])

In [158]:
y_final_6

array([0.87813971, 0.84306   , 0.98260661, ..., 0.01644177, 0.97921859,
       0.91489735])